# Conversational Interface - Chatbot with Strands SDK
In this notebook, we will build a chatbot using the Strands SDK and Amazon Bedrock Foundation Models. We use Amazon Nova Lite as our Foundation Model (FM) and implement conversation history management with Strands Agent.

## Overview

Conversational interfaces such as chatbots and virtual assistants can significantly enhance customer user experience. Modern chatbots leverage Large Language Models (LLMs) built on transformer architecture to understand context and generate human-like responses to user queries. This implementation uses the Strands SDK for simplified agent creation with LLMs on Amazon Bedrock, and includes a Streamlit demo interface.

In [1]:
import boto3
from strands import Agent
from strands.models import BedrockModel

### Set up

In [2]:
boto3_session = boto3.session.Session()
region = boto3_session.region_name

import sys
sys.path.append('../')
from util.model_selector import create_text_model_selector

# Create interactive model selector
model_selector = create_text_model_selector().display()

print("\n🎯 Select your preferred model above and run the cells below to see it in action!")


🎯 Select your preferred model above and run the cells below to see it in action!


### Run the chatbot

Set up parameters for the model and create a client

In [3]:
# Get the selected model
selected_model = model_selector.get_model_id()
model_info = model_selector.get_model_info()

print(f"Using model: {model_info['name']} ({selected_model})")
print(f"Description: {model_info['description']}")
print("\n" + "="*50 + "\n")

temperature = 0.1

Using model: Nova Lite (us.amazon.nova-lite-v1:0)
Description: Fast, cost-effective text generation




In [6]:
# Create Bedrock model with Strands SDK
bedrock_model = BedrockModel(
    model_id=selected_model,
    temperature=temperature,
    #region=region
)

The Strands Agent automatically maintains conversation history in memory during runtime, providing context across multiple interactions within a single session. For production deployments with distributed architectures or to preserve conversations across application restarts, Strands offers built-in persistence options through session managers like FileSessionManager and S3SessionManager. You can also implement custom persistence solutions using the SessionManager interface and a key-value store such as Amazon DynamoDB which enables scalable context management across distributed systems.

In [7]:
# Create a simple chatbot agent
chatbot_agent = Agent(
    system_prompt="You are a helpful assistant. Maintain conversation context and remember what users tell you.",
    model=bedrock_model,
    callback_handler=None,  # default is PrintingCallbackHandler
)

In [8]:
# Test the chatbot
response = chatbot_agent("hi - i am bob!")
print(response)

Hello, Bob! It's nice to meet you. How can I assist you today? If you have any questions or need help with something, feel free to let me know.



In [9]:
# Test conversation memory
response = chatbot_agent("whats my name?")
print(response)

Your name is Bob, as you mentioned earlier. If you ever need a reminder or have any other questions, feel free to ask! How can I assist you today, Bob?



For quick prototyping and development, you can leverage Strands Agent's built-in memory capabilities without any configuration, allowing you to focus on agent logic while conversation context is managed automatically.

### Create a Multi-Lingual Greeter Chatbot!
With Strands SDK, we can easily create specialized agents with custom system prompts. Let's create a multi-lingual translator agent:

In [10]:
# Create a multi-lingual translator agent
translator_agent = Agent(
    system_prompt="You're an assistant who speaks in French. Translate user input to French and maintain conversation context.",
    model=bedrock_model,
    callback_handler=None,  # default is PrintingCallbackHandler
)

response = translator_agent("Hi my name is John")
print(response)

Bonjour, je m'appelle John. Comment puis-je vous aider aujourd'hui ?



In [11]:
response = translator_agent("What is my name?")
print(response)

Votre nom est John. Y a-t-il autre chose que vous aimeriez discuter ?



## Streamlit Demo

Now let's create an interactive web interface using the chatbot code you've built above!

### Step 1: Add Your Code

1. Open `chatbot_demo.py`
2. Copy the bedrock model initialization and agent creation code similar to that above:
   ```python
    boto3_session = boto3.session.Session()
    region = boto3_session.region_name or "us-east-1"
    
    bedrock_model = BedrockModel(
        model_id=st.session_state.current_model,
        temperature=temperature,
        region=region
    )

    st.session_state.agent = Agent(
        system_prompt="You are a helpful AI assistant. Be conversational and maintain context throughout our chat.",
        model=bedrock_model
    )
   ```
3. Paste the code into the designated section marked `# PASTE YOUR CODE HERE`

### Step 2: Run the Demo

1. Open a terminal
2. Change the directory to where the `chatbot_demo.py` file is:

    ```
    cd /workshop/chatbots
    ```
3. Run the script with streamlit:

    ```bash
    streamlit run chatbot_demo.py
    ```
4. You will see an output like:

    ```
    You can now view your Streamlit app in your browser.

    Local URL: http://localhost:8501
    ```
5. Go to the ports tab and open the Amazon CloudFront URL next to the port (here 8501) to access the streamlit application:

    ![The **Ports** tab in the Code Editor shows URLs to ports running on the remote server](./images/forwarded_port.png)

The web interface includes:
- Model selection
- Temperature control
- Chat history with automatic context preservation
- Clear chat functionality